In [1]:
import os,sys
from sklearn import preprocessing

import json
import codecs
import yaml
import numpy as np
import pandas as pd

sys.path.append('../utils')
from cudaMetrics import *

### read metrics

In [2]:
df_app_scale = pd.read_csv('app_proflingmetrics_scaled.csv')
df_app_scale

,Unnamed: 0,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,0,rodinia_b+tree,2.309659e-02,0.000236,0.225000,0.000000,0.000000,0.050379,0.684131,0.046910,...,0.3,0.006123,0.006123,0.000000,0.631774,0.998709,9.937400e-01,0.000000,0.000000e+00,0.000000e+00
1,1,cudasdk_threadFenceReduction,2.686329e-04,0.007774,0.125000,0.000000,0.000000,0.009624,0.572213,0.034944,...,0.2,0.000500,0.000241,0.000757,1.000000,0.928899,9.696970e-01,0.149817,0.000000e+00,0.000000e+00
2,2,cudasdk_convolutionFFT2D,1.908833e-03,0.360177,1.000000,0.000000,0.000000,0.139878,0.858320,0.299212,...,0.3,0.310734,0.310734,0.000000,1.000000,0.995167,9.995885e-01,0.172893,0.000000e+00,2.685552e-03
3,3,shoc_lev1BFS,4.382805e-05,0.030554,0.154152,0.000000,0.000000,0.040641,0.822558,0.176929,...,0.1,0.001057,0.001007,0.000000,0.133829,0.122310,9.000000e-01,0.000000,0.000000e+00,0.000000e+00
4,4,rodinia_lavaMD,2.613412e-01,0.000001,0.250000,1.000000,0.000000,0.000211,0.043188,1.000000,...,0.1,0.006502,0.006502,0.000000,0.270166,0.997335,1.000000e+00,0.973673,0.000000e+00,0.000000e+00
5,5,poly_gemm,1.906970e-03,0.000079,1.000000,0.000000,0.000000,0.004901,0.667458,0.027770,...,0.8,0.899023,0.899023,0.000000,0.825273,0.978193,1.000000e+00,0.000000,0.000000e+00,1.562500e-02
6,6,rodinia_hybridsort,9.688398e-03,0.002168,1.000000,0.000000,0.000000,0.032906,0.572213,0.444358,...,0.3,0.344550,0.344543,0.096881,1.000000,0.994942,9.990244e-01,0.466314,0.000000e+00,3.417968e-03
7,7,cudasdk_MCEstimatePiInlineQ,1.696558e-04,0.001200,0.125000,0.000000,0.033385,0.000888,0.273338,0.085343,...,0.1,0.283169,0.283169,0.000000,0.125000,0.971919,2.984118e-01,0.172893,2.752721e-01,3.492460e-05
8,8,cudasdk_MCEstimatePiInlineP,2.303844e-04,0.053062,0.232143,0.000000,0.020093,0.097135,0.831953,0.113852,...,0.8,0.031818,0.031818,0.000000,0.250000,0.940444,9.587629e-01,0.172893,2.784494e-02,3.492460e-05
9,9,cudasdk_shflscan,8.838533e-04,0.119633,1.000000,0.000000,0.000000,0.067034,0.572213,0.331733,...,0.3,0.444296,0.444284,0.000000,1.000000,0.981929,1.000000e+00,0.259021,0.000000e+00,0.000000e+00


In [3]:
df_metrics = df_app_scale.drop(df_app_scale.columns[0], axis=1) # drop the 1st column

In [4]:
df_metrics

,AppName,inst_control,stall_constant_memory_dependency,gst_efficiency,inst_fp_64,local_store_transactions,stall_not_selected,global_hit_rate,stall_exec_dependency,shared_store_transactions,...,tex_fu_utilization,l2_write_throughput,l2_tex_write_throughput,atomic_transactions,gld_efficiency,sm_efficiency,l2_tex_write_hit_rate,shared_store_transactions_per_request,local_memory_overhead,flop_count_sp_fma
0,rodinia_b+tree,2.309659e-02,0.000236,0.225000,0.000000,0.000000,0.050379,0.684131,0.046910,0.000000,...,0.3,0.006123,0.006123,0.000000,0.631774,0.998709,9.937400e-01,0.000000,0.000000e+00,0.000000e+00
1,cudasdk_threadFenceReduction,2.686329e-04,0.007774,0.125000,0.000000,0.000000,0.009624,0.572213,0.034944,0.000101,...,0.2,0.000500,0.000241,0.000757,1.000000,0.928899,9.696970e-01,0.149817,0.000000e+00,0.000000e+00
2,cudasdk_convolutionFFT2D,1.908833e-03,0.360177,1.000000,0.000000,0.000000,0.139878,0.858320,0.299212,0.031246,...,0.3,0.310734,0.310734,0.000000,1.000000,0.995167,9.995885e-01,0.172893,0.000000e+00,2.685552e-03
3,shoc_lev1BFS,4.382805e-05,0.030554,0.154152,0.000000,0.000000,0.040641,0.822558,0.176929,0.000000,...,0.1,0.001057,0.001007,0.000000,0.133829,0.122310,9.000000e-01,0.000000,0.000000e+00,0.000000e+00
4,rodinia_lavaMD,2.613412e-01,0.000001,0.250000,1.000000,0.000000,0.000211,0.043188,1.000000,0.305451,...,0.1,0.006502,0.006502,0.000000,0.270166,0.997335,1.000000e+00,0.973673,0.000000e+00,0.000000e+00
5,poly_gemm,1.906970e-03,0.000079,1.000000,0.000000,0.000000,0.004901,0.667458,0.027770,0.000000,...,0.8,0.899023,0.899023,0.000000,0.825273,0.978193,1.000000e+00,0.000000,0.000000e+00,1.562500e-02
6,rodinia_hybridsort,9.688398e-03,0.002168,1.000000,0.000000,0.000000,0.032906,0.572213,0.444358,0.063761,...,0.3,0.344550,0.344543,0.096881,1.000000,0.994942,9.990244e-01,0.466314,0.000000e+00,3.417968e-03
7,cudasdk_MCEstimatePiInlineQ,1.696558e-04,0.001200,0.125000,0.000000,0.033385,0.000888,0.273338,0.085343,0.000139,...,0.1,0.283169,0.283169,0.000000,0.125000,0.971919,2.984118e-01,0.172893,2.752721e-01,3.492460e-05
8,cudasdk_MCEstimatePiInlineP,2.303844e-04,0.053062,0.232143,0.000000,0.020093,0.097135,0.831953,0.113852,0.000139,...,0.8,0.031818,0.031818,0.000000,0.250000,0.940444,9.587629e-01,0.172893,2.784494e-02,3.492460e-05
9,cudasdk_shflscan,8.838533e-04,0.119633,1.000000,0.000000,0.000000,0.067034,0.572213,0.331733,0.023229,...,0.3,0.444296,0.444284,0.000000,1.000000,0.981929,1.000000e+00,0.259021,0.000000e+00,0.000000e+00


### read ground truth

In [5]:
fastdev_dd = np.load("../05_whichgpufast/home-fastdev.npy").item()

In [6]:
fastdev_dd

{'cudasdk_BlackScholes': 1,
 'cudasdk_FDTD3d': 1,
 'cudasdk_MCEstimatePiInlineP': 1,
 'cudasdk_MCEstimatePiInlineQ': 1,
 'cudasdk_MCEstimatePiP': 1,
 'cudasdk_MCEstimatePiQ': 1,
 'cudasdk_MCSingleAsianOptionP': 1,
 'cudasdk_SobolQRNG': 1,
 'cudasdk_batchCUBLAS': 1,
 'cudasdk_binomialOptions': 1,
 'cudasdk_boxFilterNPP': 1,
 'cudasdk_c++11Cuda': 0,
 'cudasdk_concurrentKernels': 1,
 'cudasdk_convolutionFFT2D': 1,
 'cudasdk_convolutionSeparable': 1,
 'cudasdk_convolutionTexture': 1,
 'cudasdk_dct8x8': 1,
 'cudasdk_dwtHaar1D': 1,
 'cudasdk_dxtc': 1,
 'cudasdk_eigenvalues': 0,
 'cudasdk_fastWalshTransform': 1,
 'cudasdk_interval': 1,
 'cudasdk_lineOfSight': 1,
 'cudasdk_matrixMul': 1,
 'cudasdk_mergeSort': 1,
 'cudasdk_quasirandomGenerator': 1,
 'cudasdk_radixSortThrust': 1,
 'cudasdk_reduction': 1,
 'cudasdk_scalarProd': 1,
 'cudasdk_scan': 1,
 'cudasdk_segmentationTreeThrust': 1,
 'cudasdk_shflscan': 1,
 'cudasdk_simpleCUBLAS': 1,
 'cudasdk_simpleCUFFTcallback': 1,
 'cudasdk_sortingNetwor

In [7]:
targetpps = list(fastdev_dd.keys())
print len(targetpps)

75


### configure model input

In [8]:
# according to the fast device to configure the model input (delete rows not needed!)

df_metrics_cp = df_metrics.copy()

for index, row in df_metrics_cp.iterrows():
    appName = row['AppName']
    
    if not appName in targetpps:
        print("[warning] delete current row in the dataframe ({})".format(appName))
        df_metrics.drop(df_metrics[df_metrics.AppName == appName].index, inplace=True)

[warning] delete current row in the dataframe (rodinia_heartwall)


In [9]:
len(df_metrics)

75

In [10]:
len(df_metrics_cp)

76

In [11]:
# add another column to store the ground truth (best device to run the target app)
df_metrics['best_dev'] = 1 
for index, row in df_metrics.iterrows():
    appName =  row['AppName']
    fastdev = fastdev_dd[appName]
    df_metrics.loc[index, 'best_dev'] = fastdev

In [12]:
df_metrics['best_dev']

0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
20    0
21    1
22    1
23    1
24    1
25    1
26    0
27    1
28    1
29    1
     ..
46    1
47    1
48    1
49    1
50    1
51    1
52    1
53    1
54    1
55    1
56    1
57    1
58    0
59    0
60    0
61    1
62    1
63    1
64    1
65    1
66    1
67    1
68    1
69    1
70    1
71    1
72    1
73    1
74    1
75    1
Name: best_dev, Length: 75, dtype: int64

In [13]:
colist= list(df_metrics.columns)

for feat in colist:
    if feat <> 'AppName':
        print feat, min(df_metrics[feat]),  max(df_metrics[feat])

inst_control 2.3960438568e-07 0.618118706056
stall_constant_memory_dependency 0.0 1.0
gst_efficiency 0.06730769 1.0
inst_fp_64 0.0 1.0
local_store_transactions 0.0 1.0
stall_not_selected 0.0 1.0
global_hit_rate 0.0 1.0
stall_exec_dependency 0.000304812847343 1.0
shared_store_transactions 0.0 1.0
branch_efficiency 0.31204091086 1.0
warp_nonpred_execution_efficiency 0.356115436714 1.0
inst_bit_convert 0.0 0.0382817345766
gst_throughput 0.0 1.0
sysmem_read_throughput 1.0 1.0
issue_slot_utilization 0.00815540633557 1.0
shared_efficiency 0.0 1.0
shared_load_transactions 0.0 1.0
flop_count_dp_fma 0.0 1.0
shared_store_throughput 0.0 1.0
inst_executed 7.68376966032e-08 0.542769543592
achieved_occupancy 1.12238026243e-05 1.0
stall_inst_fetch 0.00155414753913 1.0
dram_read_transactions 0.0 1.0
dram_write_transactions 3.03537547999e-07 1.0
stall_sync 0.0 1.0
gld_transactions_per_request 0.0 1.0
single_precision_fu_utilization 0.0 1.0
gst_requested_throughput 0.0 1.0
sysmem_write_utilization 1.0 1

### change small number in df_metrics to 0

In [14]:
colist= list(df_metrics.columns)
for feat in colist:
    if feat <> 'AppName': # not the 1st column
        df_metrics[df_metrics[feat] < 1e-6] = 0

### generate input x and label y

In [15]:
def gen_model_input(df_dataset):
    df_X = df_dataset.drop(['AppName', 'best_dev'], axis=1)
    df_y = df_dataset['best_dev']
    df_y = df_y.astype('int64')  # convert obj to int
    return df_X, df_y


df_X, df_y = gen_model_input(df_metrics)

In [19]:
df_X.isnull().values.any()

False

In [ ]:
df_X.isin

In [17]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFold
from sklearn import metrics

#------------------------------------------------------------------------------
# Neural Networks 
#------------------------------------------------------------------------------
def GreedySearch_MLP(df_X, df_y, fold_k=3):
    from sklearn.neural_network import MLPClassifier
    
    # 3 fold, stratifiedKFold
    kf = StratifiedKFold(n_splits=fold_k, random_state=314159, shuffle=True)
    #
    # parameters
    # 
    parameters = {'hidden_layer_sizes': [(30,30,30), (60,60,60), (100,100,100)],
                  'activation': ('logistic', 'tanh', 'relu', 'identity'),
                  'solver': ('lbfgs', 'sgd', 'adam'),
                  'alpha': [1., 0.1, 0.01, 0.001]}
        
    param_combo = []
    for v1 in parameters['hidden_layer_sizes']:
        for v2 in parameters['activation']:
            for v3 in parameters['solver']:
                for v4 in parameters['alpha']:
                    param_combo.append({'hidden_layer_sizes': v1,
                                        'activation': v2,
                                        'solver': v3,
                                        'alpha': v4})    
    #for combo in param_combo: print combo
    
    
    #
    # go through each combo, find out the error rate
    #
    param_combo_dd = {}
    minError = 1
    for current_param in param_combo:
        #print current_param

        #
        # run Kfold
        #
        error_list = []
        for train_index, test_index in kf.split(df_X, df_y):
            #print("TRAIN:", train_index, "TEST:", test_index)
            X_train, X_test = df_X.loc[train_index], df_X.loc[test_index]
            y_train, y_test = df_y.loc[train_index], df_y.loc[test_index]

            clsfy = MLPClassifier(hidden_layer_sizes=current_param['hidden_layer_sizes'],
                                  activation=current_param['activation'],
                                  solver=current_param['solver'],
                                  alpha=current_param['alpha'],
                                  max_iter=1000) # max 1K iterations
            
            clsfy.fit(X_train, y_train)
            err = metrics.mean_absolute_error(y_test, clsfy.predict(X_test))
            error_list.append(err)

        #
        # update error 
        #
        
        # use mean error for comparision
        eval_error = np.mean(error_list)
        print eval_error
        
        if eval_error < minError:
            minError = eval_error
            
        param_combo_dd[eval_error] = current_param
 
    #
    # print final results
    #
    
    #print param_combo_dd
    return minError, param_combo_dd[minError]



In [18]:
nn_error, nn_params = GreedySearch_MLP(df_X, df_y, fold_k=5)
print nn_error, nn_params

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').